### PROMPT 1

I need to check what campaign was effective by analyzing a csv data as follows:
date,firstSessionDate,firstUserSourceMedium,firstUserManualTerm,firstUserManualAdContent,newUsers,conversions,totalUsers,transactions,checkouts,totalPurchasers,engagedSessions,userEngagementDuration,bounceRate,pageReferrer,landingPagePlusQueryString
```csv
20230103,20230103,(direct) / (none),(not set),(not set),145,150,145,0,0,0,112,1941,0.24324324324324326,,/product/rentfull
20230103,20230103,google / cpc,(not set),(not set),65,65,65,0,0,0,35,525,0.46153846153846156,https://googleads.g.doubleclick.net/,/
20230201,20230201,danggn / da,(not set),40대 여성,63,64,63,0,0,0,4,30,0.93650793650793651,,/product/trial
20230131,20230131,danggn / da,(not set),반값,53,53,53,0,0,0,12,145,0.77358490566037741,,/product/trial
20230208,20230208,danggn / da,(not set),40대 여성,48,48,48,0,0,0,3,14,0.93877551020408168,,/product/trial
```

would you recommend what is the way to check it with python code? and if you need to draw a chart use plotly express. 


In [1]:
import pandas as pd
import plotly.express as px
df = pd.read_csv('./report.csv')
df['conversion_rate'] = df['conversions'] / df['newUsers']

grouped_df = df.groupby('firstUserSourceMedium')['conversion_rate'].mean().reset_index()
fig = px.bar(grouped_df, x='firstUserSourceMedium', y='conversion_rate', title='Average Conversion Rate by Campaign')
fig.show()



### PROMTP 2 

the data is stored in report.csv file so don'w show the data in the example code and to make the code read the report.csv file. 
I would be much better if you merge the fields firstUserSourceMedium and firstUserManualAdContent and use it as key of campaign.
lastly exclude rows if the firstUserSourcMedium filed contains direct or referal. 
transpose the chart so that current x axis goes to y axis. 


In [4]:
import pandas as pd
import plotly.express as px

# Read the CSV data from the file
df = pd.read_csv('report.csv')

# Merge the firstUserSourceMedium and firstUserManualAdContent fields
df['campaign'] = df['firstUserSourceMedium'] + ' - ' + df['firstUserManualAdContent']

# Exclude rows containing 'direct' or 'referral' in the firstUserSourceMedium field
df = df[~df['firstUserSourceMedium'].str.contains('direct|referral', case=False)]

# Calculate the conversion rate for each campaign
df['conversion_rate'] = df['conversions'] / df['totalUsers']

# Group the data by the campaign, and calculate the average conversion rate
grouped_df = df.groupby('campaign')['conversion_rate'].mean().reset_index()

# Visualize the average conversion rate using Plotly Express (transpose the chart)
fig = px.bar(grouped_df, x='conversion_rate', y='campaign', title='Average Conversion Rate by Campaign', orientation='h')
fig.show()

### PROMPT 3


I need to also check how long each campain trigers the landing page by checking and how was the strengh by checking of size of newUsers per day. So draw a scatter chart and express of nubmers of newUser to the marker size. Surely, the campaign key should be in y axis and date is x axis.




In [46]:
import pandas as pd
import plotly.express as px

# Read the CSV data from the file
df = pd.read_csv('report.csv')

# Merge the firstUserSourceMedium and firstUserManualAdContent fields
df['campaign'] = df['firstUserSourceMedium'] + ' - ' + df['firstUserManualAdContent']

# Exclude rows containing 'direct' or 'referral' in the firstUserSourceMedium field
df = df[~df['firstUserSourceMedium'].str.contains('direct|referral', case=False)]

# Convert the date column to a datetime object
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# Create a scatter chart with the campaign key on the y-axis, date on the x-axis, and the number of new users as the marker size
fig = px.scatter(df, x='date', y='campaign', size='newUsers', title='New Users by Date and Campaign', height=1500)
fig.show()
# userEngagementDuration

In [48]:
import pandas as pd
import plotly.express as px

# Read the CSV data from the file
df = pd.read_csv('report.csv')

# Merge the firstUserSourceMedium and firstUserManualAdContent fields
df['campaign'] = df['firstUserSourceMedium'] + ' - ' + df['firstUserManualAdContent']

# Exclude rows containing 'direct' or 'referral' in the firstUserSourceMedium field
df = df[~df['firstUserSourceMedium'].str.contains('direct|referral', case=False)]

# Convert the date column to a datetime object
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# Create a scatter chart with the campaign key on the y-axis, date on the x-axis, and the number of new users as the marker size
fig = px.scatter(df, x='date', y='campaign', size='userEngagementDuration', title='New Users by Date and Campaign', height=1500, color_discrete_sequence=['red'],)
fig.show()
# userEngagementDuration

### PROMPT 4
good! it would be better to draw it with timeline of plotly express instead of scatter chart.  each y tick has different color and show the number of total users as darkness of color. when hover the bar on timeline chart show number of total users and engagedSessions, userEngagementDuration with unit which is seconds, and bounceRate. 
Since there are a lot of unique campaigns you have to define a custom color scale with enough colors to cover all unique campaigns.


### PROMTP 5
I think we need to consider two more things. 
1. use plotly.figure_factory and  create_gantt
2. we re-structure pandas dataframe to fit to this chart. 
   to make it possible we have to make groups of bar based on observed consecutive date totalUsers is more than 0.
Can you make it ? if you have something to clarify, ask me. 

In [17]:
import pandas as pd
import plotly.figure_factory as ff
import seaborn as sns

# Read the CSV data from the file
df = pd.read_csv('report.csv')

# Merge the firstUserSourceMedium and firstUserManualAdContent fields
df['campaign'] = df['firstUserSourceMedium'] + ' - ' + df['firstUserManualAdContent']

# Exclude rows containing 'direct' or 'referral' in the firstUserSourceMedium field
df = df[~df['firstUserSourceMedium'].str.contains('direct|referral', case=False)]

# Convert the date column to a datetime object
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# Create an empty DataFrame with the required columns
gantt_df = pd.DataFrame(columns=['Task', 'Start', 'Finish', 'totalUsers'])

# Iterate over each campaign
for campaign in df['campaign'].unique():
    campaign_df = df[df['campaign'] == campaign].sort_values(by='date')
    
    start_date = None
    end_date = None
    total_users = 0
    
    for index, row in campaign_df.iterrows():
        if row['totalUsers'] > 0:
            if start_date is None:
                start_date = row['date']
            end_date = row['date']
            total_users += row['totalUsers']
        else:
            if start_date is not None:
                new_row = pd.DataFrame({'Task': [campaign], 'Start': [start_date], 'Finish': [end_date], 'totalUsers': [total_users]})
                gantt_df = pd.concat([gantt_df, new_row], ignore_index=True)
                start_date = None
                end_date = None
                total_users = 0

    if start_date is not None:
        new_row = pd.DataFrame({'Task': [campaign], 'Start': [start_date], 'Finish': [end_date], 'totalUsers': [total_users]})
        gantt_df = pd.concat([gantt_df, new_row], ignore_index=True)

# Create a custom color scale
color_scale = sns.color_palette("husl", n_colors=len(gantt_df['Task'].unique())).as_hex()

# Create a dictionary mapping each unique campaign to a color
color_dict = {campaign: color for campaign, color in zip(gantt_df['Task'].unique(), color_scale)}

# Create the Gantt chart using plotly.figure_factory.create_gantt
fig = ff.create_gantt(gantt_df, colors=color_dict, index_col='Task', show_colorbar=True, bar_width=0.5, showgrid_x=True, showgrid_y=True, height=1500)

fig.show()


### PROMPT 6
Can show more information on the hover popup with totalUsers, total days, average bounce rate, average userEngagedDuration (which is sum of userEngagementDuration/ sum of engagedSessions) of each campaign? 
Plust, make the hover popup shows upper side of mouse position where previous campaign resides above the current bar. When you add custom hover information, please keep the previous code as much as you can. especially must use ff.create_gantt". 

In [44]:
import pandas as pd
import plotly.figure_factory as ff
import seaborn as sns

# Read the CSV data from the file
df = pd.read_csv('report.csv')

# Exclude rows containing 'direct' or 'referral' in the firstUserSourceMedium field
df = df[~df['firstUserSourceMedium'].str.contains('direct|referral', case=False)]

# Merge the firstUserSourceMedium and firstUserManualAdContent fields
df['campaign'] = df['firstUserSourceMedium'] + ' - ' + df['firstUserManualAdContent']

# Convert the date column to a datetime object
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# Create an empty DataFrame with the required columns
gantt_df = pd.DataFrame(columns=['Task', 'Start', 'Finish', 'totalUsers'])

# Iterate over each campaign
for campaign in df['campaign'].unique():
    campaign_df = df[df['campaign'] == campaign].sort_values(by='date')
    
    start_date = None
    end_date = None
    total_users = 0
    
    for index, row in campaign_df.iterrows():
        if row['totalUsers'] > 0:
            if start_date is None:
                start_date = row['date']
            end_date = row['date']
            total_users += row['totalUsers']
        else:
            if start_date is not None:
                new_row = pd.DataFrame({'Task': [campaign], 'Start': [start_date], 'Finish': [end_date], 'totalUsers': [total_users]})
                gantt_df = pd.concat([gantt_df, new_row], ignore_index=True)
                start_date = None
                end_date = None
                total_users = 0

    if start_date is not None:
        new_row = pd.DataFrame({'Task': [campaign], 'Start': [start_date], 'Finish': [end_date], 'totalUsers': [total_users]})
        gantt_df = pd.concat([gantt_df, new_row], ignore_index=True)
display(gantt_df)
# Create a custom color scale
color_scale = sns.color_palette("husl", n_colors=len(gantt_df['Task'].unique())).as_hex()

# Create a dictionary mapping each unique campaign to a color
color_dict = {campaign: color for campaign, color in zip(gantt_df['Task'].unique(), color_scale)}

# Create custom hover text for each row
hover_text = []
for index, row in gantt_df.iterrows():
    hover_text.append(f"<b>{row['Task']}</b><br>Total Users: {row['totalUsers']}")

# Create the Gantt chart using plotly.figure_factory.create_gantt
fig = ff.create_gantt(gantt_df, colors=color_dict, index_col='Task', show_colorbar=True, bar_width=0.5, showgrid_x=True, showgrid_y=True, height=1500)

# Update hover text
for trace, hover in zip(fig['data'], hover_text):
    trace['hovertemplate'] = hover

fig.update_layout(
    hovermode='y unified',
    height=1500,
    yaxis=dict(title='Campaigns'),
    xaxis=dict(title='Date'),
    plot_bgcolor='white',
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=False  # Add this line to remove the legend
)
fig.show()


,Task,Start,Finish,totalUsers
0,google / cpc - (not set),2023-01-02 00:00:00,2023-03-16 00:00:00,595
1,danggn / da - 40대 여성,2023-02-01 00:00:00,2023-02-09 00:00:00,192
2,danggn / da - 반값,2023-01-31 00:00:00,2023-02-01 00:00:00,71
3,danggn / da - 전문,2023-02-06 00:00:00,2023-02-17 00:00:00,147
4,danggn / da - 40대 남성,2023-02-01 00:00:00,2023-02-06 00:00:00,89
...,...,...,...,...
59,kakaoview / mobile - product,2023-02-07 00:00:00,2023-02-07 00:00:00,1
60,recognition / instagram - drink_on_linen,2023-02-07 00:00:00,2023-02-10 00:00:00,5
61,recognition / instagram - jelly_on_linen,2023-02-07 00:00:00,2023-02-08 00:00:00,3
62,recognition / instagram - snack_on_linen,2023-02-07 00:00:00,2023-02-09 00:00:00,4
